# Lab Three: Extending Logistic Regression

Team: Miro Ronac, Kirk Watson, Brandon Vincitore

## 1. Preparation and Overview

### 1.1 Task and Use-case

This data can be useful in identifying prediabetes or diabetes in patients and assisting doctors with making accurate observations from a variety of health indicators.

Every year, the CDC collects data from a health-related telephone survey called the Behavioral Risk Factor Surveillance System. The data gathered from these surveys include information on “health-related risk behaviors, chronic health conditions, and use of preventive services.” This dataset focuses on responses from 2015 and diabetes, a “prevalent chronic disease in the United States.”

Ultimately, the ability to identify a patient with prediabetes or diabetes with increased efficiency and accuracy is the intention of analyzing this dataset. With this capability, a diabetes diagnosis can be reached at a faster rate compared to when a human makes the diagnosis. Diabetes is extremely common in the US as about 1 in 10 Americans have diabetes, and [about 1 in 5 people with diabetes don’t know they have it](https://www.cdc.gov/diabetes/library/spotlights/diabetes-facts-stats.html#:~:text=37.3%20million%20Americans%E2%80%94about%201,t%20know%20they%20have%20it.). In addition, 1 in 3 Americans have prediabetes, and [more than 8 in 10 adults with prediabetes don’t know they have it](https://www.cdc.gov/diabetes/library/spotlights/diabetes-facts-stats.html#:~:text=37.3%20million%20Americans%E2%80%94about%201,t%20know%20they%20have%20it.). Using this classifier, patients that might be at risk of diabetes can be brought to a doctor’s attention at a higher rate allowing for earlier medical care and attention.

Dataset Source: https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset

### 1.2 Dataset Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('diabetes_012_health_indicators_BRFSS2015.csv')
print('Size of dataset:', df.shape[0])
df.head()

Size of dataset: 253680


,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [2]:
# Creating target features by modifying first column in original dataframe such that we have 3 features consisting of binary values for
# no diabetes, prediabetes, and diabetes where 0 is False and 1 is True
target_array = np.zeros((len(df),4))
for i in range(len(df)):
    target_array[i,0] = df['Diabetes_012'].values[i]
for i in range(len(target_array)):
    # no diabetes
    if target_array[i,0] == 0:
        target_array[i,1] = 1
    # prediabetes
    if target_array[i,0] == 1:
        target_array[i,2] = 1
    # diabetes
    if target_array[i,0] == 2:
        target_array[i,3] = 1

# Adding new target columns to original dataframe
target_columns = ['NoDiabetes', 'PreDiabetes', 'Diabetes']
for i in range(target_array.shape[1]-1):
    df.insert(i, target_columns[i], target_array[:,1:][:,i], True)
df_target = df.drop('Diabetes_012', axis=1)

columns = list(df_target.columns)
for col in columns:
    df_target[col] = df_target[col].astype(int)
    
df_target.drop(df_target.tail(248000).index,inplace = True)
print('Size of dataset:', df_target.shape[0])
df_target.head()

Size of dataset: 5680


,NoDiabetes,PreDiabetes,Diabetes,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1,0,0,1,1,1,40,1,0,0,...,1,0,5,18,15,1,0,9,4,3
1,1,0,0,0,0,0,25,1,0,0,...,0,1,3,0,0,0,0,7,6,1
2,1,0,0,1,1,1,28,0,0,0,...,1,1,5,30,30,1,0,9,4,8
3,1,0,0,1,0,1,27,0,0,0,...,1,0,2,0,0,0,0,11,3,6
4,1,0,0,1,1,1,24,0,0,0,...,1,0,2,3,0,0,0,11,5,4


### 1.3 Dataset Training and Testing Split

In [3]:
targets = ['NoDiabetes', 'PreDiabetes', 'Diabetes']
for col in targets:
    columns.remove(col)
    
# Splitting dataset
from sklearn.model_selection import train_test_split as tts

train, test = tts(df_target, test_size=.20, random_state=42, shuffle=True)

X_test  = test[columns].to_numpy()
X_train = train[columns].to_numpy()
y_test = test[targets].to_numpy()
y_train = train[targets].to_numpy()

With a large dataset of over 250,000 instances, an 80/20 split is sufficient. Our classifier has plenty of data to use for training and also has plenty of data to use for training. We could comformtably move our split to 75/25 or 70/30 if we desired more opportunities to test our classifier. With such a large dataset, we could also divide the dataset with an additional validation set. With a validation set, we can use this set for more frequent model evaulations and save the testing dataset for a final unbiased evaluation. 

UPDATE: We reduced the size of the dataset to allow for reasonable runtime. With 5680 instances, the dataset is large enough to the above to still be valid. However, we do not have as much freedom to possible create a validation set.

## 2. Modeling
### Binary Logistic Regression

In [4]:
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta=0.01, iterations=50, optimization='sd', regularization='none', C=0):
        
        self.eta = eta
        self.iters = iterations
        self.opt = optimization
        self.reg = regularization
        self.C = C
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self, X, add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) # return the actual prediction

In [5]:
from scipy.special import expit
from numpy.linalg import pinv

# inherit from base class
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    # private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    # creating ability to choose optimization technique
    def _get_gradient(self,X,y):
        
        gradient = None
        if self.opt == 'sd': 
            gradient = self.steepest_descent
        elif self.opt == 'sgd': 
            gradient = self.stochastic_gradient_descent
        elif self.opt == 'newton': 
            gradient = self.newton
        else:
            print('No optimization chosen')
        return gradient(X,y)
    
    def steepest_descent(self,X,y):
    
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
    
    def stochastic_gradient_descent(self,X,y):
        
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
    
    def newton(self,X,y):
        
        g = self.predict_proba(X,add_bias=False).ravel() # get sigmoid value for all classes
        
        hessian = X.T @ np.diag(g*(1-g)) @ X - 2 * self.C # calculate the hessian

        ydiff = y-g # get y difference
        
        gradient = np.sum(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += self.C * self._get_reg_gradient()
        return pinv(hessian) @ gradient
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # regularization methods
    def _get_reg_gradient(self):
        
        if self.reg == 'none':
            return self.w_[1:]
        elif self.reg == 'L1':
            return np.sign(self.w_[1:])
        elif self.reg == 'L2':
            return -2 * self.w_[1:]
        elif self.reg == 'L1_L2':
            return -2 * self.w_[1:] + np.sign(self.w_[1:])    
        
    def fit(self, X, y):
        
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        for _ in range(int(self.iters)):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate

### Logistic Regression Class

In [34]:
class LogisticRegression:
    
    def __init__(self, eta, iterations, optimization, regularization, C=0):
        
        self.eta = eta
        self.iters = iterations
        self.opt = optimization
        self.reg = regularization
        self.C = C
        self.encodings = {}

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
    
    def fit(self,X,y):
        
        self.classifiers_ = []
        for i in range(len(y[0,:])):
            blr = BinaryLogisticRegression(self.eta, self.iters, self.opt, self.reg, self.C)
            blr.fit(X,y[:,i])
            self.classifiers_.append(blr)
            
        # saving weights
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
    
    def predict_proba(self,X):
        
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier
    
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row

### Testing the logistic regression

In [7]:
# Validating training dataset
from sklearn.metrics import accuracy_score
lr = LogisticRegression(optimization='sd',eta=0.9, regularization='none', iterations=10, C=0.01)
for i,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,i],yhat), " - ", col)

Accuracy of Testing Dataset:  0.18397887323943662  -  NoDiabetes
Accuracy of Testing Dataset:  0.9894366197183099  -  PreDiabetes
Accuracy of Testing Dataset:  0.8265845070422535  -  Diabetes


### Increasing Cost

In [8]:
lr = LogisticRegression(optimization='sd',eta=0.9, regularization='none', iterations=10, C=0.99)
for i,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,i],yhat), " - ", col)

Accuracy of Testing Dataset:  0.8160211267605634  -  NoDiabetes
Accuracy of Testing Dataset:  0.01056338028169014  -  PreDiabetes
Accuracy of Testing Dataset:  0.17341549295774647  -  Diabetes


### Changing Optimization Technique

In [9]:
lr = LogisticRegression(optimization='newton',eta=0.9, regularization='none', iterations=10, C=0.01)
for i,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,i],yhat), " - ", col)

Accuracy of Testing Dataset:  0.1505281690140845  -  NoDiabetes
Accuracy of Testing Dataset:  0.9242957746478874  -  PreDiabetes
Accuracy of Testing Dataset:  0.7931338028169014  -  Diabetes


### 2.2 Optimizing the classifier
#### Steepest Descent

In [10]:
import operator
from collections import OrderedDict
#find best eta
performance=dict()
for i in range(1,99,4):
    lr = LogisticRegression(optimization="sd",eta=i/100, regularization='none', iterations=50, C=.01)
    for j,col in zip(range(len(y_test[0,:])), targets):
        lr.fit(X_train, y_train)
        yhat = lr.predict(X_test)
        performance[accuracy_score(y_test[:,j],yhat)] = i/100
        
top_performance = OrderedDict(sorted(performance.items(),reverse=True)[:5])
print('Top Eta Performance')
for key in top_performance:
    print('Accuracy:',key,'Eta:', top_performance[key])
    
best_eta = top_performance[list(top_performance.keys())[0]]
#find best C
performance=dict()
for i in range(1,99,4):
    lr = LogisticRegression(optimization="sd",eta=best_eta, regularization='none', iterations=50, C=i/100)
    for j,col in zip(range(len(y_test[0,:])), targets):
        lr.fit(X_train, y_train)
        yhat = lr.predict(X_test)
        performance[accuracy_score(y_test[:,j],yhat)] = i/1000
        
top_performance = OrderedDict(sorted(performance.items(),reverse=True)[:5])
print()
print('Top C Performance')
for key in top_performance:
    print('Accuracy:',key,'C:', top_performance[key])
    
best_c = top_performance[list(top_performance.keys())[0]]
#find best regularization term
reg_terms = ['none','L1','L2','L1_L2']
performance=dict()
for reg in reg_terms:
    lr = LogisticRegression(optimization="sd",eta=best_eta, regularization=reg, iterations=50, C=best_c)
    for j,col in zip(range(len(y_test[0,:])), targets):
        lr.fit(X_train, y_train)
        yhat = lr.predict(X_test)
        performance[accuracy_score(y_test[:,j],yhat)] = reg
        
top_performance = OrderedDict(sorted(performance.items(),reverse=True)[:5])
print()
print('Top Regularization Performance')
for key in top_performance:
    print('Accuracy:',key,'Regularization:', top_performance[key])
    
best_reg = top_performance[list(top_performance.keys())[0]]
print()
print("Optimized Config - Eta:",best_eta,"C:",best_c,"Regularization:",best_reg)

Top Eta Performance
Accuracy: 0.9894366197183099 Eta: 0.97
Accuracy: 0.988556338028169 Eta: 0.65
Accuracy: 0.9859154929577465 Eta: 0.77
Accuracy: 0.8829225352112676 Eta: 0.45
Accuracy: 0.8371478873239436 Eta: 0.49

Top C Performance
Accuracy: 0.9894366197183099 C: 0.005
Accuracy: 0.9876760563380281 C: 0.009
Accuracy: 0.9577464788732394 C: 0.017
Accuracy: 0.8265845070422535 C: 0.005
Accuracy: 0.8257042253521126 C: 0.009

Top Regularization Performance
Accuracy: 0.9894366197183099 Regularization: L1_L2
Accuracy: 0.8265845070422535 Regularization: L1_L2
Accuracy: 0.18397887323943662 Regularization: L1_L2

Optimized Config - Eta: 0.97 C: 0.005 Regularization: L1_L2


In [11]:
#optimized performance
lr = LogisticRegression(optimization="sd",eta=best_eta, regularization=best_reg, iterations=50, C=best_c)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = reg
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,j],yhat), " - ", col)

Accuracy of Testing Dataset:  0.18397887323943662  -  NoDiabetes
Accuracy of Testing Dataset:  0.9894366197183099  -  PreDiabetes
Accuracy of Testing Dataset:  0.8265845070422535  -  Diabetes


#### Stochastic Gradient Descent

In [12]:
#find best eta
performance=dict()
for i in range(1,99,4):
    lr = LogisticRegression(optimization="sgd",eta=i/100, regularization='none', iterations=50, C=.01)
    for j,col in zip(range(len(y_test[0,:])), targets):
        lr.fit(X_train, y_train)
        yhat = lr.predict(X_test)
        performance[accuracy_score(y_test[:,j],yhat)] = i/100
        
top_performance = OrderedDict(sorted(performance.items(),reverse=True)[:5])
print('Top Eta Performance')
for key in top_performance:
    print('Accuracy:',key,'Eta:', top_performance[key])
    
best_eta = top_performance[list(top_performance.keys())[0]]
#find best C
performance=dict()
for i in range(1,99,4):
    lr = LogisticRegression(optimization="sgd",eta=best_eta, regularization='none', iterations=50, C=i/100)
    for j,col in zip(range(len(y_test[0,:])), targets):
        lr.fit(X_train, y_train)
        yhat = lr.predict(X_test)
        performance[accuracy_score(y_test[:,j],yhat)] = i/1000
        
top_performance = OrderedDict(sorted(performance.items(),reverse=True)[:5])
print()
print('Top C Performance')
for key in top_performance:
    print('Accuracy:',key,'C:', top_performance[key])
    
best_c = top_performance[list(top_performance.keys())[0]]
#find best regularization term
reg_terms = ['none','L1','L2','L1_L2']
performance=dict()
for reg in reg_terms:
    lr = LogisticRegression(optimization="sgd",eta=best_eta, regularization=reg, iterations=50, C=best_c)
    for j,col in zip(range(len(y_test[0,:])), targets):
        lr.fit(X_train, y_train)
        yhat = lr.predict(X_test)
        performance[accuracy_score(y_test[:,j],yhat)] = reg
        
top_performance = OrderedDict(sorted(performance.items(),reverse=True)[:5])
print()
print('Top Regularization Performance')
for key in top_performance:
    print('Accuracy:',key,'Regularization:', top_performance[key])
    
best_reg = top_performance[list(top_performance.keys())[0]]
print()
print("Optimized Config - Eta:",best_eta,"C:",best_c,"Regularization:",best_reg)

Top Eta Performance
Accuracy: 0.9894366197183099 Eta: 0.97
Accuracy: 0.983274647887324 Eta: 0.33
Accuracy: 0.9797535211267606 Eta: 0.17
Accuracy: 0.9762323943661971 Eta: 0.61
Accuracy: 0.9683098591549296 Eta: 0.25

Top C Performance
Accuracy: 0.9894366197183099 C: 0.097
Accuracy: 0.8265845070422535 C: 0.057
Accuracy: 0.8257042253521126 C: 0.025
Accuracy: 0.8160211267605634 C: 0.097
Accuracy: 0.7913732394366197 C: 0.005

Top Regularization Performance
Accuracy: 0.9894366197183099 Regularization: none
Accuracy: 0.988556338028169 Regularization: L1_L2
Accuracy: 0.8265845070422535 Regularization: L1_L2
Accuracy: 0.8177816901408451 Regularization: L1
Accuracy: 0.7200704225352113 Regularization: L1

Optimized Config - Eta: 0.97 C: 0.097 Regularization: none


In [25]:
#optimized performance
lr = LogisticRegression(optimization="sgd",eta=best_eta, regularization=best_reg, iterations=50, C=best_c)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = reg
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,j],yhat), " - ", col)

Accuracy of Testing Dataset:  0.8133802816901409  -  NoDiabetes
Accuracy of Testing Dataset:  0.9894366197183099  -  PreDiabetes
Accuracy of Testing Dataset:  0.8265845070422535  -  Diabetes


#### Newton's method

In [14]:
#values were tested at a more sparse range to make up for extended runtime with newton's method
#find best eta
performance=dict()
#eta .01
lr = LogisticRegression(optimization="newton",eta=.01, regularization='none', iterations=50, C=.01)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = .01
#eta .25
lr = LogisticRegression(optimization="newton",eta=.25, regularization='none', iterations=50, C=.01)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = .25
#eta .5  
lr = LogisticRegression(optimization="newton",eta=.5, regularization='none', iterations=50, C=.01)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = .5
#eta .75
lr = LogisticRegression(optimization="newton",eta=.75, regularization='none', iterations=50, C=.01)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = .75
#eta .99
lr = LogisticRegression(optimization="newton",eta=.99, regularization='none', iterations=50, C=.01)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = .99
        
top_performance = OrderedDict(sorted(performance.items(),reverse=True)[:5])
print('Top Eta Performance')
for key in top_performance:
    print('Accuracy:',key,'Eta:', top_performance[key])
    
best_eta = top_performance[list(top_performance.keys())[0]]
#find best C
#c .01
lr = LogisticRegression(optimization="newton",eta=best_eta, regularization='none', iterations=50, C=.01)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = .01
#c .25
lr = LogisticRegression(optimization="newton",eta=best_eta, regularization='none', iterations=50, C=.25)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = .25
#c .5  
lr = LogisticRegression(optimization="newton",eta=best_eta, regularization='none', iterations=50, C=.5)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = .5
#c .75
lr = LogisticRegression(optimization="newton",eta=best_eta, regularization='none', iterations=50, C=.75)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = .75
#c .99
lr = LogisticRegression(optimization="newton",eta=best_eta, regularization='none', iterations=50, C=.99)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = .99
        
top_performance = OrderedDict(sorted(performance.items(),reverse=True)[:5])
print()
print('Top C Performance')
for key in top_performance:
    print('Accuracy:',key,'C:', top_performance[key])
    
best_c = top_performance[list(top_performance.keys())[0]]
#find best regularization term
reg_terms = ['none','L1','L2','L1_L2']
performance=dict()
for reg in reg_terms:
    lr = LogisticRegression(optimization="newton",eta=best_eta, regularization=reg, iterations=50, C=best_c)
    for j,col in zip(range(len(y_test[0,:])), targets):
        lr.fit(X_train, y_train)
        yhat = lr.predict(X_test)
        performance[accuracy_score(y_test[:,j],yhat)] = reg
        
top_performance = OrderedDict(sorted(performance.items(),reverse=True)[:5])
print()
print('Top Regularization Performance')
for key in top_performance:
    print('Accuracy:',key,'Regularization:', top_performance[key])
    
best_reg = top_performance[list(top_performance.keys())[0]]
print()
print("Optimized Config - Eta:",best_eta,"C:",best_c,"Regularization:",best_reg)

Top Eta Performance
Accuracy: 0.9551056338028169 Eta: 0.01
Accuracy: 0.9242957746478874 Eta: 0.99
Accuracy: 0.8107394366197183 Eta: 0.01
Accuracy: 0.7931338028169014 Eta: 0.99
Accuracy: 0.16549295774647887 Eta: 0.01

Top C Performance
Accuracy: 0.9551056338028169 C: 0.5
Accuracy: 0.954225352112676 C: 0.99
Accuracy: 0.9524647887323944 C: 0.75
Accuracy: 0.9242957746478874 C: 0.99
Accuracy: 0.8107394366197183 C: 0.5

Top Regularization Performance
Accuracy: 0.9551056338028169 Regularization: L1_L2
Accuracy: 0.8107394366197183 Regularization: L1_L2
Accuracy: 0.16549295774647887 Regularization: L1_L2

Optimized Config - Eta: 0.01 C: 0.5 Regularization: L1_L2


In [15]:
#optimized performance
lr = LogisticRegression(optimization="newton",eta=best_eta, regularization=best_reg, iterations=50, C=best_c)
for j,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    performance[accuracy_score(y_test[:,j],yhat)] = reg
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,j],yhat), " - ", col)

Accuracy of Testing Dataset:  0.16549295774647887  -  NoDiabetes
Accuracy of Testing Dataset:  0.9551056338028169  -  PreDiabetes
Accuracy of Testing Dataset:  0.8107394366197183  -  Diabetes


When finding the optimal performance for each optimization technique, we can see that stochastic gradient descent has the most reliable results with an eta value of .97, a C value of .097, and using both L1 and L2 regularizations. Our approach to selecting these parameters is justified because we investigated each combination of the parameters. We were able to find reliable eta and C values by testing a range of values between .01 and .99. While we couldn't test a larger range of values due to runtime constraints, the utilized range provided a reasonable variety of tests. After finding these values, each regularization term tested to find which worked best.

Our approach to finding parameters did not lead to data snooping because we used eta and C values after iterating through a range of values to find the highest accuracy.

### 2.3 Scikit-learn Comparison

In [30]:
import time
#optimized performance for stochastic gradient descent
lr = LogisticRegression(optimization="sgd",eta=.97, regularization='L1_L2', iterations=50, C=.097)
total_time=0
for j,col in zip(range(len(y_test[0,:])), targets):
    start_time = time.time()
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    total_time += (time.time() - start_time)
    performance[accuracy_score(y_test[:,j],yhat)] = reg
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,j],yhat), " - ", col)
    
print("Average runtime: {}s".format(float(total_time)/3))

Accuracy of Testing Dataset:  0.7570422535211268  -  NoDiabetes
Accuracy of Testing Dataset:  0.9894366197183099  -  PreDiabetes
Accuracy of Testing Dataset:  0.8265845070422535  -  Diabetes
Average runtime: 0.0023299853006998696s


In [32]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='liblinear')
total_time=0
for j,col in zip(range(len(y_test[0,:])), targets):
    start_time = time.time()
    clf.fit(X_train, y_train[:,j])
    clf.predict(X_test)
    clf.predict_proba(X_test)
    total_time += (time.time() - start_time)
    print("Accuracy of Testing Dataset: ", clf.score(X_test,y_test[:,j]), ' - ', col)

print("Average runtime: {}s".format(float(total_time)/3))

Accuracy of Testing Dataset:  0.8204225352112676  -  NoDiabetes
Accuracy of Testing Dataset:  0.9894366197183099  -  PreDiabetes
Accuracy of Testing Dataset:  0.8265845070422535  -  Diabetes
Average runtime: 0.01701521873474121s


Compared to our "best" logistic regression optimization procedure: stochastic gradient descent, the procedure used in scikit-learn has a better classification performance. Our classifier using stochastic gradient descent was able to outperform scikit-learn in regards to training time.

## 3. Deployment

Scikit Learn has a comparatively superior implementation for logistic regression for multiple reasons. One of the main reasons is that it is written in C++, as opposed to Python, which is essentially a wrapper for C in most instances. Moreover, since Scikit is an external libray, the user, for the most part, need not worry about the various packages and dependencies necessary to effectively implement the programming.

## 4. Exceptional Work
###### Option 1: Optimizing logistic regression by implementing MSE and using Newton's method to update the values of  "w".

In [35]:
from sklearn.metrics import brier_score_loss

# inherit from base class
class BinaryLogisticRegression_MSE(BinaryLogisticRegressionBase):
    # private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    # creating ability to choose optimization technique
    def _get_gradient(self,X,y):
        
        if self.opt == 'sd': 
            gradient = self.steepest_descent
        elif self.opt == 'sgd': 
            gradient = self.stochastic_gradient_descent
        elif self.opt == 'newton': 
            gradient = self.newton
        else:
            print('No optimization chosen')
        return gradient(X,y)
    
    def steepest_descent(self,X,y):
    
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
    
    def stochastic_gradient_descent(self,X,y):
        
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
    
    def newton(self,X,y):
        
        g = self.predict_proba(X,add_bias=False).ravel() # get sigmoid value for all classes
        hessian = X.T @ np.diag(g*(1-g)) @ X - 2 * self.C # calculate the hessian
        
        mse = np.zeros((len(y,)))
        for i in range(len(y)):
            mse[i] = brier_score_loss(y, g)
        
        gradient = (np.sum(X * mse[:,np.newaxis], axis=0)) / len(mse)

        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += self.C * self._get_reg_gradient()
        return pinv(hessian) @ gradient
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # regularization methods
    def _get_reg_gradient(self):
        
        if self.reg == 'none':
            return self.w_[1:]
        elif self.reg == 'L1':
            return np.sign(self.w_[1:])
        elif self.reg == 'L2':
            return -2 * self.w_[1:]
        elif self.reg == 'L1_L2':
            return -2 * self.w_[1:] + np.sign(self.w_[1:])    
        
    def fit(self, X, y):
        
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        for _ in range(int(self.iters)):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate

In [36]:
class LogisticRegression_MSE:
    
    def __init__(self, eta, iterations, optimization, regularization, C=0):
        
        self.eta = eta
        self.iters = iterations
        self.opt = optimization
        self.reg = regularization
        self.C = C
        self.encodings = {}

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
    
    def fit(self,X,y):
        
        self.classifiers_ = []
        for i in range(len(y[0,:])):
            blr = BinaryLogisticRegression_MSE(self.eta, self.iters, self.opt, self.reg, self.C)
            blr.fit(X,y[:,i])
            self.classifiers_.append(blr)
            
        # saving weights
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
    
    def predict_proba(self,X):
        
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier
    
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row

In [37]:
# Validating training dataset
from sklearn.metrics import accuracy_score
lr = LogisticRegression(optimization='newton',eta=0.9, regularization='none', iterations=10, C=0)
for i,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,i],yhat), " - ", col)

Accuracy of Testing Dataset:  0.1505281690140845  -  NoDiabetes
Accuracy of Testing Dataset:  0.9242957746478874  -  PreDiabetes
Accuracy of Testing Dataset:  0.7931338028169014  -  Diabetes


In [38]:
# Validating training dataset
from sklearn.metrics import accuracy_score
lr = LogisticRegression_MSE(optimization='newton',eta=0.9, regularization='none', iterations=10, C=0)
for i,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,i],yhat), " - ", col)

Accuracy of Testing Dataset:  0.8160211267605634  -  NoDiabetes
Accuracy of Testing Dataset:  0.01056338028169014  -  PreDiabetes
Accuracy of Testing Dataset:  0.17341549295774647  -  Diabetes


Here, we attempt to implement mean square error (MSE) to optimize logistic regression. We use scikit's brier score function so that we can calculate a mean squared difference between the predicted probability and the actual outcome. It is clear that there is a noticeable drop in the accuracy of 2 of the 3 target features, therefore indicating that MSE is not better than binary cross entropy when optimizing a logistic regression operator. This is likely due to the fact that the MSE loss function is not convex. As a result, you are more likely to get stuck at a local minimum instead of reaching the global minimum, which drastically reduces the accuracy of the results. 